In [24]:
import tensorflow as tf
import keras.api._v2.keras as keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models
from keras.applications import MobileNetV2
from PIL import Image
import numpy as np

In [25]:
img_size = (224, 224)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=True
)

Found 474 images belonging to 3 classes.
Found 117 images belonging to 3 classes.


In [26]:
base_model = MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

In [27]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [28]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [29]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=12
)

Epoch 1/12
30/30 [==============================] - 8s 228ms/step - loss: 0.3314 - accuracy: 0.8734 - val_loss: 0.0749 - val_accuracy: 0.9658
Epoch 2/12
30/30 [==============================] - 6s 189ms/step - loss: 0.0292 - accuracy: 0.9958 - val_loss: 0.0319 - val_accuracy: 0.9915
Epoch 3/12
30/30 [==============================] - 6s 185ms/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.0255 - val_accuracy: 0.9915
Epoch 4/12
30/30 [==============================] - 6s 199ms/step - loss: 0.0156 - accuracy: 0.9958 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 5/12
30/30 [==============================] - 6s 183ms/step - loss: 0.0094 - accuracy: 0.9958 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 6/12
30/30 [==============================] - 6s 185ms/step - loss: 0.0110 - accuracy: 0.9979 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 7/12
30/30 [==============================] - 6s 185ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 

In [30]:
model.save("plushie_model_4.h5")

In [31]:
from keras.models import load_model

model = load_model("plushie_model_4.h5")

In [41]:
from PIL import Image
import numpy as np
import tensorflow as tf

img_size = (224, 224)
img_path = "testWEINER.png"

img = Image.open(img_path).convert("RGB")
img = img.resize(img_size)
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
predicted_probabilities = predictions[0]

predicted_class_index = np.argmax(predicted_probabilities)
confidence = predicted_probabilities[predicted_class_index] * 100

class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class_index]

print(f"Predicted class: {predicted_label}")
print(f"Confidence: {confidence:.2f}%")


1/1 [==============================] - 0s 24ms/step
Predicted class: WEINER
Confidence: 99.72%
